# Trabalho Final de Sudoku utilizando o LTNtorch

Este notebook resolve a **Questão 1**, **Questão 2** e **Questão 3** do trabalho: classificar se um tabuleiro de Sudoku fechado é válido com base nas regras do jogo, usando lógica de primeira ordem com a biblioteca `LTNtorch`, classificar um tabuleiro inicial / aberto (usar heurísticas) e indicar para um tabuleiro aberto quais heurísticas são mais recomendadas.

In [ ]:
!pip install git+https://github.com/tommasocarraro/LTNtorch

  Cloning https://github.com/tommasocarraro/LTNtorch to /tmp/pip-req-build-gwp856wy
  Running command git clone --filter=blob:none --quiet https://github.com/tommasocarraro/LTNtorch /tmp/pip-req-build-gwp856wy
  Resolved https://github.com/tommasocarraro/LTNtorch to commit d1bd98169cc2121f8cdd25ff99901e4589923c95
  Preparing metadata (setup.py) ... done


**Importação com o Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importação de Bibliotecas Necessárias**

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import ltn

# **Questão 1**

Consiste em classificar um tabuleiro de Sudoku fechado (com todas as células preenchidas) como válido ou inválido.


1. Predicado "Different": Define a lógica do predicado Different. Este predicado, implementado como uma rede neural simples (DifferentModel), calcula a "diferença" entre dois números. Sua saída é próxima de 1 se os números são distintos e próxima de 0 se são iguais, permitindo que o LTN avalie a violação de restrições.

2. Carregar CSV: A função carregar_tabuleiro_csv lê o tabuleiro do Sudoku de um arquivo CSV, convertendo-o em um tensor PyTorch para processamento.

3. Geração dos Axiomas: A função gerar_axiomas constrói as restrições lógicas do Sudoku. Para cada linha, coluna e bloco do tabuleiro, ela cria axiomas utilizando o predicado Different para assegurar que todos os números já preenchidos sejam únicos dentro de cada uma dessas unidades. Apenas valores não-zero são considerados para a geração dos axiomas.

4. Classificador: A função classificar_tabuleiro avalia a validade do tabuleiro. Ela calcula a satisfação geral de todos os axiomas gerados, pegando o valor mínimo de verdade entre eles. Se esse valor mínimo de satisfação for igual ou superior a um limiar (0.95 por padrão), o tabuleiro é considerado válido; caso contrário, é inválido.

5. Execução Principal: O bloco if __name__ == "__main__": carrega um tabuleiro de exemplo e utiliza o classificador para determinar e imprimir se o tabuleiro é válido ou inválido de acordo com as regras do Sudoku.

In [ ]:
# ---------- Predicado "Different" ----------
class DifferentModel(nn.Module):
    def forward(self, x, y):
        return 1.0 - torch.exp(-100 * torch.abs(x - y))

Different = ltn.Predicate(DifferentModel())

# ---------- Carregar CSV ----------
def carregar_tabuleiro_csv(path):
    df = pd.read_csv(path, header=None)
    return torch.tensor(df.values, dtype=torch.float32)

# ---------- Geração dos axiomas ----------
def gerar_axiomas(tabuleiro):
    n = tabuleiro.shape[0]
    axiomas = []

    # ---------- Linhas ----------
    for i in range(n):
        for j1 in range(n):
            for j2 in range(j1 + 1, n):
                v1_val = tabuleiro[i][j1].item()
                v2_val = tabuleiro[i][j2].item()
                if v1_val != 0 and v2_val != 0:
                    v1 = ltn.Constant(torch.tensor([v1_val]))
                    v2 = ltn.Constant(torch.tensor([v2_val]))
                    axiomas.append(Different(v1, v2))

    # ---------- Colunas ----------
    for j in range(n):
        for i1 in range(n):
            for i2 in range(i1 + 1, n):
                v1_val = tabuleiro[i1][j].item()
                v2_val = tabuleiro[i2][j].item()
                if v1_val != 0 and v2_val != 0:
                    v1 = ltn.Constant(torch.tensor([v1_val]))
                    v2 = ltn.Constant(torch.tensor([v2_val]))
                    axiomas.append(Different(v1, v2))

    # ---------- Blocos ----------
    b = int(n ** 0.5)
    for bi in range(0, n, b):
        for bj in range(0, n, b):
            celulas = []
            for i in range(bi, bi + b):
                for j in range(bj, bj + b):
                    val = tabuleiro[i][j].item()
                    if val != 0:
                        celulas.append(ltn.Constant(torch.tensor([val])))
            for k1 in range(len(celulas)):
                for k2 in range(k1 + 1, len(celulas)):
                    axiomas.append(Different(celulas[k1], celulas[k2]))

    return axiomas

# ---------- Classificador ----------
def classificar_tabuleiro(tabuleiro, limiar=0.95):
    axiomas = gerar_axiomas(tabuleiro)
    if not axiomas:
        return 1  # tabuleiro vazio é considerado válido
    verdades = torch.stack([ax.value for ax in axiomas]).squeeze()
    score = verdades.min()
    return 1 if score.item() >= limiar else 0

# ---------- Main ----------
if __name__ == "__main__":
    caminho = "/content/drive/MyDrive/trabalho-final-IA/tabuleiro4x4-invalido.csv"  # coloque o nome/caminho do seu arquivo
    tabuleiro = carregar_tabuleiro_csv(caminho)
    res = classificar_tabuleiro(tabuleiro)
    print("1 -✅ Tabuleiro VÁLIDO!" if res else "0 - ❌ Tabuleiro INVÁLIDO!")


0 - ❌ Tabuleiro INVÁLIDO!


# **Questão 2**

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import ltn
import random

# ---------- Modelo MLP treinável ----------
class ValidMoveModel(nn.Module):
    def __init__(self, n=4):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x)

# ---------- Predicado LTN ----------
model = ValidMoveModel()
ValidMove = ltn.Predicate(model)

# ---------- Carrega tabuleiro ----------
def carregar_tabuleiro_csv(path):
    df = pd.read_csv(path, header=None)
    return torch.tensor(df.values, dtype=torch.float32)

# ---------- Verifica se (i,j,v) é válida ----------
def jogada_valida(tabuleiro, i, j, v):
    n = tabuleiro.shape[0]
    if v in tabuleiro[i] or v in tabuleiro[:, j]:
        return False
    b = int(n ** 0.5)
    bi, bj = i - i % b, j - j % b
    bloco = tabuleiro[bi:bi+b, bj:bj+b].flatten()
    return v not in bloco

# ---------- Gera dados de treino ----------
def gerar_dados_treino(tabuleiro, amostras=300):
    n = tabuleiro.shape[0]
    X = []
    Y = []
    for _ in range(amostras):
        i = random.randint(0, n-1)
        j = random.randint(0, n-1)
        v = random.randint(1, n)
        if tabuleiro[i][j] != 0:
            continue
        x = torch.tensor([i, j, v], dtype=torch.float32)
        y = 1.0 if jogada_valida(tabuleiro, i, j, v) else 0.0
        X.append(x)
        Y.append(torch.tensor([y]))
    return torch.stack(X), torch.stack(Y)

# ---------- Treina a MLP ----------
def treinar_modelo(tabuleiro):
    X, Y = gerar_dados_treino(tabuleiro)
    const_X = ltn.Variable("x", X)
    label_Y = ltn.Constant(Y)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(100):
        optimizer.zero_grad()
        predicoes = ValidMove(const_X)
        loss = nn.BCELoss()(predicoes.value, Y.squeeze())
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")
    print("✅ Treinamento concluído.")

# ---------- Avalia jogadas ----------
def sugerir_jogadas(tabuleiro):
    n = tabuleiro.shape[0]
    sugestoes = []
    for i in range(n):
        for j in range(n):
            if tabuleiro[i][j] != 0:
                continue
            for v in range(1, n+1):
                entrada = torch.tensor([[i, j, v]], dtype=torch.float32)
                prob = ValidMove(ltn.Constant(entrada)).value.item()
                if prob > 0.7:  # ajustável
                    sugestoes.append(((i, j, v), prob))
    return sorted(sugestoes, key=lambda x: -x[1])

# ---------- Main ----------
if __name__ == "__main__":
    caminho = "/content/drive/MyDrive/trabalho-final-IA/tabuleiro4x4-parcial-2.csv"
    tabuleiro = carregar_tabuleiro_csv(caminho)

    print("🔁 Treinando rede para prever jogadas válidas...")
    treinar_modelo(tabuleiro)

    print("🎯 Jogadas com alta probabilidade de serem válidas:")
    sugestoes = sugerir_jogadas(tabuleiro)
    for (i, j, v), prob in sugestoes:
        print(f" -> Posição ({i},{j}) valor {v} = {prob:.2f}")


🔁 Treinando rede para prever jogadas válidas...
Epoch 0: Loss = 0.7086
Epoch 20: Loss = 0.4929
Epoch 40: Loss = 0.4351
Epoch 60: Loss = 0.3850
Epoch 80: Loss = 0.3393
✅ Treinamento concluído.
🎯 Jogadas com alta probabilidade de serem válidas:
 -> Posição (3,0) valor 1 = 1.00
 -> Posição (3,2) valor 1 = 0.96
 -> Posição (2,0) valor 1 = 0.81
 -> Posição (2,3) valor 1 = 0.81


## Questão (todos os cenários)

Este trabalho final foca na projeção e implementação de uma solução para o Sudoku utilizando LTNTorch, visando "aprender os conceitos" conforme as questões propostas. Para a:

* Questão 1, que classifica tabuleiros fechados, utilizou-se o framework Logic Tensor Networks (LTN) com um predicado Different para verificar a satisfação das restrições lógicas do Sudoku.
* Já as Questões 2 e 3, que abordam tabuleiros abertos, foram implementadas com base em NumPy para classificar enigmas como "sem solução" ou "solução possível" e para indicar as heurísticas mais recomendadas, como MRV.

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import ltn
import sys
import os
import copy # Para copy.deepcopy

# --- Constantes Globais (serão definidas dinamicamente por set_board_size) ---
BOARD_SIZE = None
BLOCK_SIZE = None

# --- Predicado LTN "Different" ---
class DifferentModel(nn.Module):
    def forward(self, x, y):
        return 1.0 - torch.exp(-100 * torch.abs(x - y))

Different = ltn.Predicate(DifferentModel())

**Funções Auxiliares Comuns**

In [ ]:
# --- Funções Auxiliares Comuns ---
def set_board_size(board_np):
    global BOARD_SIZE, BLOCK_SIZE
    BOARD_SIZE = board_np.shape[0]
    BLOCK_SIZE = int(np.sqrt(BOARD_SIZE))
    if BLOCK_SIZE * BLOCK_SIZE != BOARD_SIZE:
        print(f"Erro: O tamanho do tabuleiro {BOARD_SIZE} não é um quadrado perfeito (necessário para Sudoku).")
        return False
    return True

def carregar_tabuleiro_csv(path):
    print(f"\nLendo o tabuleiro do arquivo '{path}'...")
    try:
        df = pd.read_csv(path, header=None)
        board_np = df.values.astype(int)
        board_tensor = torch.tensor(board_np, dtype=torch.float32)

        if not set_board_size(board_np):
            return None, None

        if board_np.shape[0] != board_np.shape[1]:
            print(f"ERRO: O tabuleiro no arquivo '{path}' tem dimensões não quadradas {board_np.shape}.")
            return None, None

        if np.any(board_np < 0) or np.any(board_np > BOARD_SIZE):
            print(f"ERRO: O tabuleiro contém números fora do intervalo válido [0, {BOARD_SIZE}].")
            return None, None

        print("Tabuleiro Lido:\n", board_np)
        return board_np, board_tensor

    except FileNotFoundError:
        print(f"ERRO: Arquivo '{path}' não encontrado.")
        return None, None
    except ValueError:
        print(f"ERRO: O arquivo '{path}' não parece ser um CSV válido para o tabuleiro (contém não-números ou formato incorreto).")
        return None, None
    except Exception as e:
        print(f"ERRO inesperado ao carregar o tabuleiro: {e}")
        return None, None

def get_empty_cells(board_np):
    cells = []
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board_np[r, c] == 0:
                cells.append((r, c))
    return cells

def violates_constraint(board_np, move):
    row, col, digit = move

    if not (1 <= digit <= BOARD_SIZE):
        return True

    if digit in board_np[row, :]:
        return True
    if digit in board_np[:, col]:
        return True
    start_row, start_col = (row // BLOCK_SIZE) * BLOCK_SIZE, (col // BLOCK_SIZE) * BLOCK_SIZE
    if digit in board_np[start_row:start_row + BLOCK_SIZE, start_col:start_col + BLOCK_SIZE]:
        return True
    return False

**Lógica da Questão 1 - Classificar Tabuleiro Fechado (usando LTN)**

Nesta questão, o objetivo foi classificar um tabuleiro de Sudoku que está completamente preenchido (fechado), independentemente de seu tamanho ser 4×4 ou 9×9. A solução implementada utiliza o framework Logic Tensor Networks (LTN) para verificar a validade do tabuleiro. Para isso, foi definido um predicado Different em LTN, que mede o quão "diferentes" são dois valores.

Em seguida, foram gerados axiomas lógicos para as restrições do Sudoku (linhas, colunas e blocos), utilizando esse predicado para garantir que todos os números em cada unidade (linha, coluna ou bloco) sejam únicos. O sistema calcula a satisfação geral desses axiomas. Se a satisfação estiver acima de um limiar predefinido (0.99), o tabuleiro é classificado como 1 (corretamente preenchido); caso contrário, é classificado como 0 (violou alguma restrição).

In [ ]:
# --- Lógica da Questão 1: Classificar Tabuleiro Fechado (usando LTN Different) ---
def gerar_axiomas_fechado(tabuleiro_tensor):
    n = tabuleiro_tensor.shape[0]
    b = int(n ** 0.5)
    axiomas = []

    for i in range(n):
        filled_values_in_row = []
        for j in range(n):
            val = tabuleiro_tensor[i, j].item()
            if val != 0:
                filled_values_in_row.append(ltn.Constant(torch.tensor([val])))

        for k1 in range(len(filled_values_in_row)):
            for k2 in range(k1 + 1, len(filled_values_in_row)):
                axiomas.append(Different(filled_values_in_row[k1], filled_values_in_row[k2]))

    for j in range(n):
        filled_values_in_col = []
        for i in range(n):
            val = tabuleiro_tensor[i, j].item()
            if val != 0:
                filled_values_in_col.append(ltn.Constant(torch.tensor([val])))

        for k1 in range(len(filled_values_in_col)):
            for k2 in range(k1 + 1, len(filled_values_in_col)):
                axiomas.append(Different(filled_values_in_col[k1], filled_values_in_col[k2]))

    for bi in range(0, n, b):
        for bj in range(0, n, b):
            filled_values_in_block = []
            for i in range(bi, bi + b):
                for j in range(bj, bj + b):
                    val = tabuleiro_tensor[i, j].item()
                    if val != 0:
                        filled_values_in_block.append(ltn.Constant(torch.tensor([val])))

            for k1 in range(len(filled_values_in_block)):
                for k2 in range(k1 + 1, len(filled_values_in_block)):
                    axiomas.append(Different(filled_values_in_block[k1], filled_values_in_block[k2]))

    return axiomas

def classificar_cenario1_tabuleiro_fechado(tabuleiro_np, tabuleiro_tensor, limiar=0.99):
    print("\n--- CENÁRIO 1: CLASSIFICAR TABULEIRO FECHADO (Questão 1) ---")

    if np.any(tabuleiro_np == 0):
        print("Aviso: O tabuleiro contém células vazias (0). A classificação verificará apenas a validade dos números preenchidos.")

    axiomas = gerar_axiomas_fechado(tabuleiro_tensor)

    if not axiomas:
        score = torch.tensor(1.0)
    else:
        verdades = torch.stack([ax.value for ax in axiomas]).squeeze()
        score = verdades.min()

    print(f"Satisfação geral dos axiomas LTN: {score.item():.4f}")

    if score.item() >= limiar:
        print(f"Classificação para CENÁRIO 1: 1 - ✅ Tabuleiro VÁLIDO (Satisfação >= {limiar})!")
        return 1
    else:
        print(f"Classificação para CENÁRIO 1: 0 - ❌ Tabuleiro INVÁLIDO (Satisfação < {limiar})!")
        return 0

**Lógica da Questão 2 - Classificar Tabuleiro Aberto e Cenários (usando Numpy)**

Para esta questão, o foco foi analisar um tabuleiro de Sudoku que possui células vazias (aberto). A classificação principal determina se o enigma:


1) Não possui solução: Ocorre se houver um numeral válido (1 a 4 para 4×4 ou 1 a 9 para 9×9) que não pode ser colocado em nenhuma célula vazia restante sem violar as restrições do Sudoku.


2) Possui solução possível: Classificação aplicada caso a situação anterior não se verifique.
___


Além da classificação, o sistema indica os numerais que têm maior probabilidade de levar a um impasse (situação sem solução) ou de manter uma solução possível, analisando:

a. Movimentos em um passo: Simula-se a colocação de cada numeral em cada célula vazia, avaliando o estado do tabuleiro resultante (se leva a "sem solução" ou "solução possível").

b. Movimentos em dois passos: A análise é estendida para sequências de dois movimentos, identificando padrões que resultam em impasses ou mantêm a viabilidade da solução.

In [ ]:
# --- Lógica da Questão 2: Classificar Tabuleiro Aberto e Cenários (usando Numpy) ---
def check_sem_solucao(board_np):
    empty_cells = get_empty_cells(board_np)
    if not empty_cells:
        return False

    for digit_to_check in range(1, BOARD_SIZE + 1):
        can_place_digit = False
        for cell in empty_cells:
            move = (cell[0], cell[1], digit_to_check)
            if not violates_constraint(board_np, move):
                can_place_digit = True
                break

        if not can_place_digit:
            print(f"DIAGNÓSTICO: O dígito '{digit_to_check}' não pode ser colocado em nenhuma célula vazia restante.")
            return True
    return False

def evaluate_one_move(board_np):
    empty_cells = get_empty_cells(board_np)
    moves_leading_to_sem_solucao = []
    moves_maintaining_solucao_possivel = []

    for row, col in empty_cells:
        for digit in range(1, BOARD_SIZE + 1):
            move = (row, col, digit)

            if violates_constraint(board_np, move):
                continue

            copied_board = np.copy(board_np)
            copied_board[row, col] = digit

            if check_sem_solucao(copied_board):
                moves_leading_to_sem_solucao.append(move)
            else:
                moves_maintaining_solucao_possivel.append(move)
    return moves_leading_to_sem_solucao, moves_maintaining_solucao_possivel

def evaluate_two_moves(board_np):
    empty_cells = get_empty_cells(board_np)
    two_moves_leading_to_sem_solucao = []
    two_moves_maintaining_solucao_possivel = []

    for r1, c1 in empty_cells:
        for d1 in range(1, BOARD_SIZE + 1):
            first_move = (r1, c1, d1)

            if violates_constraint(board_np, first_move):
                continue

            board_after_first_move = copy.deepcopy(board_np)
            board_after_first_move[r1, c1] = d1

            if check_sem_solucao(board_after_first_move):
                two_moves_leading_to_sem_solucao.append((first_move, "DIAG: 1º passo leva a Sem Solução"))
                continue

            empty_cells_after_first_move = get_empty_cells(board_after_first_move)

            if not empty_cells_after_first_move:
                continue

            for r2, c2 in empty_cells_after_first_move:
                if (r1, c1) == (r2, c2): # Prevents putting a second digit in the same spot just filled by the first move
                    continue

                for d2 in range(1, BOARD_SIZE + 1):
                    second_move = (r2, c2, d2)

                    if violates_constraint(board_after_first_move, second_move):
                        continue

                    board_after_two_moves = copy.deepcopy(board_after_first_move)
                    board_after_two_moves[r2, c2] = d2

                    if check_sem_solucao(board_after_two_moves):
                        two_moves_leading_to_sem_solucao.append((first_move, second_move))
                    else:
                        two_moves_maintaining_solucao_possivel.append((first_move, second_move))

    return two_moves_leading_to_sem_solucao, two_moves_maintaining_solucao_possivel

def classificar_cenario2_tabuleiro_aberto(board_np):
    print("\n--- CENÁRIO 2: CLASSIFICAR TABULEIRO ABERTO (Questão 2) ---")

    empty_cells = get_empty_cells(board_np)
    if not empty_cells:
        print("O tabuleiro está completo. Não há células vazias para analisar.")
        print("Classificação para CENÁRIO 2: O tabuleiro está preenchido, não se aplica 'aberto'.")
        return 0

    is_sem_solucao = check_sem_solucao(board_np)

    if is_sem_solucao:
        print("Classificação para CENÁRIO 2: 1) Sem Solução")
    else:
        print("Classificação para CENÁRIO 2: 2) Solução Possível")

        print("\n--- CENÁRIO 2a: AVALIAÇÃO DE MOVIMENTOS EM UM PASSO ---")
        moves_leading_to_sem_solucao_one_step, moves_maintaining_solucao_possivel_one_step = evaluate_one_move(board_np)
        print(f"Movimentos que levam a 'Sem Solução': {moves_leading_to_sem_solucao_one_step}")
        print(f"Movimentos que mantêm 'Solução Possível': {moves_maintaining_solucao_possivel_one_step}")

        if moves_leading_to_sem_solucao_one_step:
            digits_causing_impasse = {}
            for r, c, d in moves_leading_to_sem_solucao_one_step:
                digits_causing_impasse[d] = digits_causing_impasse.get(d, 0) + 1
            sorted_digits_impasse = sorted(digits_causing_impasse.items(), key=lambda item: item[1], reverse=True)
            print(f"Numerais com maior probabilidade de levar a 'Sem Solução' (em 1 movimento): {sorted_digits_impasse}")
        else:
            print("Nenhum movimento em um passo leva a 'Sem Solução'.")

        print("\n--- CENÁRIO 2b: AVALIAÇÃO DE SEQUÊNCIAS DE DOIS MOVIMENTOS ---")
        two_moves_leading_to_sem_solucao, two_moves_maintaining_solucao_possivel = evaluate_two_moves(board_np)
        print(f"Sequências de dois movimentos que levam a 'Sem Solução': {two_moves_leading_to_sem_solucao}")
        print(f"Sequências de dois movimentos que mantêm 'Solução Possível': {two_moves_maintaining_solucao_possivel}")

        if two_moves_leading_to_sem_solucao:
            digits_causing_impasse_two_steps = {}
            for move1, move2 in two_moves_leading_to_sem_solucao:
                digits_causing_impasse_two_steps[move1[2]] = digits_causing_impasse_two_steps.get(move1[2], 0) + 1
            sorted_digits_impasse_two_steps = sorted(digits_causing_impasse_two_steps.items(), key=lambda item: item[1], reverse=True)
            print(f"Numerais com maior probabilidade de levar a 'Sem Solução' (em 2 movimentos, considerando o 1º dígito): {sorted_digits_impasse_two_steps}")
        else:
            print("Nenhuma sequência de dois movimentos leva a 'Sem Solução'.")

**Lógica da Questão 3 - Indicar Heurísticas Mais Recomendadas (usando Numpy)**

Nesta questão, o objetivo foi indicar heurísticas relevantes para a resolução de um tabuleiro de Sudoku aberto. Foram implementadas e comparadas duas heurísticas clássicas de problemas de satisfação de restrições:

* Heurística MRV (Minimum Remaining Values): Identifica as células vazias que possuem o menor número de opções válidas para preenchimento. Priorizar essas células pode ajudar a detectar impasses mais cedo.

* Heurística do Dígito Mais Restrito: Foca nos numerais que podem ser colocados no menor número de posições válidas no tabuleiro. Alocar esses dígitos pode evitar problemas futuros.

In [ ]:
# --- Lógica da Questão 3: Indicar Heurísticas Mais Recomendadas (usando Numpy) ---
def initialize_binary_variables(board_np):
    binary_vars = np.zeros((BOARD_SIZE, BOARD_SIZE, BOARD_SIZE + 1), dtype=int)
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            num = board_np[i, j]
            if num != 0:
                if 1 <= num <= BOARD_SIZE:
                    binary_vars[i, j, num] = 1
    return binary_vars

def is_valid_for_heuristic(binary_vars_3d, row, col, num):
    if np.sum(binary_vars_3d[row, :, num]) > 0:
        return False
    if np.sum(binary_vars_3d[:, col, num]) > 0:
        return False
    block_row_start = (row // BLOCK_SIZE) * BLOCK_SIZE
    block_col_start = (col // BLOCK_SIZE) * BLOCK_SIZE
    if np.sum(binary_vars_3d[block_row_start:block_row_start + BLOCK_SIZE,
                             block_col_start:block_col_start + BLOCK_SIZE, num]) > 0:
        return False
    return True

def heuristic_mrv(board_np_original):
    binary_vars = initialize_binary_variables(board_np_original)
    empty_cells = get_empty_cells(board_np_original)

    cell_options = []
    for r, c in empty_cells:
        options = []
        for num in range(1, BOARD_SIZE + 1):
            if is_valid_for_heuristic(binary_vars, r, c, num):
                options.append(num)
        cell_options.append(((r, c), options))

    cell_options.sort(key=lambda x: len(x[1]))
    return cell_options

def heuristic_most_constrained_digit(board_np_original):
    binary_vars = initialize_binary_variables(board_np_original)
    empty_cells = get_empty_cells(board_np_original)

    digit_spaces = {}
    for num in range(1, BOARD_SIZE + 1):
        count = 0
        for r, c in empty_cells:
            if is_valid_for_heuristic(binary_vars, r, c, num):
                count += 1
        digit_spaces[num] = count

    sorted_digits = sorted(digit_spaces.items(), key=lambda x: x[1])
    return sorted_digits

def classificar_cenario3_indicar_heuristicas(board_np):
    print("\n--- CENÁRIO 3: INDICAR HEURÍSTICAS MAIS RECOMENDADAS (Questão 3) ---")

    empty_cells = get_empty_cells(board_np)
    if not empty_cells:
        print("O tabuleiro está completo. Não há células vazias para aplicar heurísticas.")
        return

    print("\n--- Heurística MRV (Minimum Remaining Values) ---")
    mrv_results = heuristic_mrv(board_np)
    if mrv_results:
        print("Células com o menor número de opções válidas (mais restritas):")
        for (r, c), opts in mrv_results[:min(5, len(mrv_results))]:
            print(f"   Célula ({r},{c}) → {len(opts)} possibilidade(s): {opts}")
    else:
        print("Não há células vazias para aplicar MRV.")

    print("\n--- Heurística 'Dígito Mais Restrito' ---")
    most_constrained_digit_results = heuristic_most_constrained_digit(board_np)
    if most_constrained_digit_results:
        print("Dígitos com o menor número de posições válidas no tabuleiro (mais restritos):")
        for digit, count in most_constrained_digit_results[:min(4, len(most_constrained_digit_results))]:
            print(f"   Dígito {digit} → {count} posição(ões) possível(is)")
    else:
        print("Não há dígitos para analisar em células vazias.")

    print("\nConsiderações sobre as heurísticas:")
    print("A heurística MRV foca na célula mais difícil de preencher. Preenche-la primeiro pode expor um impasse cedo.")
    print("A heurística do Dígito Mais Restrito foca no dígito mais difícil de alocar. Alocá-lo pode evitar problemas futuros.")
    print("Ambas são úteis para guiar a busca em problemas de satisfação de restrições.")

    print("\n--- Pergunta Final: Resolução de Sudoku com LTN (Restrições + Heurísticas) ---")
    print("Sim, é possível resolver o Sudoku usando apenas LTN combinando restrições e heurísticas.")
    print("Para isso, o modelo 'DummyModel' precisaria ser substituído por uma Rede Neural real (ex: MLP).")
    print("Essa rede aprenderia a 'confiança' de cada número em cada célula.")
    print("Os axiomas LTN (como 'Different', e outros para 'HasNumber' e 'IsFilled' para células vazias)")
    print("formariam a função de perda. Otimizadores (SGD, Adam) ajustariam os pesos da rede para maximizar")
    print("a satisfação geral dos axiomas. As heurísticas seriam incorporadas como cláusulas LTN adicionais")
    print("para guiar o aprendizado da rede, priorizando certos tipos de atribuições.")
    print("Por exemplo, penalizando soluções onde uma célula tem muitas opções ou favorecendo a alocação de dígitos raros.")
    print("Após o treinamento, o tabuleiro seria resolvido selecionando os números com maior confiança pela rede.")

**Função Principal (Main) para Execução**

In [ ]:
# --- Função Principal (Main) para Executar os Cenários no Colab ---
if __name__ == "__main__":
    # --- DEFINA A PASTA NO SEU GOOGLE DRIVE AQUI ---
    # Este é o caminho para a pasta que contém seus arquivos CSV de Sudoku
    # Exemplo: Se sua pasta é 'trabalho-final-IA' dentro de 'Meu Drive'
    # directory_path = "/content/drive/MyDrive/trabalho-final-IA/"
    # Se você tiver uma subpasta 'test_boards' dentro de 'trabalho-final-IA', seria:
    directory_path = "/content/drive/MyDrive/trabalho-final-IA/" # AJUSTE ESTE CAMINHO CONFORME SEU DRIVE

    if not os.path.isdir(directory_path):
        print(f"ERRO: O caminho '{directory_path}' não é um diretório válido ou não foi montado corretamente.")
        print("Certifique-se de que o Google Drive está montado e o caminho está correto.")
        sys.exit(1)

    print(f"\nProcessando arquivos CSV na pasta '{directory_path}'...")

    # Lista todos os arquivos CSV na pasta especificada
    csv_files = [f for f in os.listdir(directory_path) if f.endswith(".csv")]

    if not csv_files:
        print(f"Nenhum arquivo CSV encontrado na pasta: '{directory_path}'")
        sys.exit(0)

    for filename in csv_files:
        file_path = os.path.join(directory_path, filename)

        # Carrega o tabuleiro como numpy array e torch tensor
        board_np, board_tensor = carregar_tabuleiro_csv(file_path)

        if board_np is not None and board_tensor is not None:
            print(f"\n==== ANÁLISE PARA O ARQUIVO: {filename} ====")

            # --- Executa o Cenário 1 (Questão 1) ---
            classificar_cenario1_tabuleiro_fechado(board_np, board_tensor)

            # --- Executa o Cenário 2 (Questão 2) ---
            # A classificação de tabuleiro aberto e a avaliação de movimentos
            # só fazem sentido se o tabuleiro não estiver completamente preenchido.
            if np.any(board_np == 0):
                classificar_cenario2_tabuleiro_aberto(board_np)
                # --- Executa o Cenário 3 (Questão 3) ---
                classificar_cenario3_indicar_heuristicas(board_np)
            else:
                print("\n--- CENÁRIO 2 & 3: Não aplicável. O tabuleiro está completo. ---")
        else:
            print(f"\n==== Pulando arquivo: {filename} devido a erros de carregamento/validação ====")


Processando arquivos CSV na pasta '/content/drive/MyDrive/trabalho-final-IA/'...

Lendo o tabuleiro do arquivo '/content/drive/MyDrive/trabalho-final-IA/tabuleiro4x4-valido.csv'...
Tabuleiro Lido:
 [[1 2 3 4]
 [3 4 1 2]
 [2 1 4 3]
 [4 3 2 1]]

==== ANÁLISE PARA O ARQUIVO: tabuleiro4x4-valido.csv ====

--- CENÁRIO 1: CLASSIFICAR TABULEIRO FECHADO (Questão 1) ---
Satisfação geral dos axiomas LTN: 1.0000
Classificação para CENÁRIO 1: 1 - ✅ Tabuleiro VÁLIDO (Satisfação >= 0.99)!

--- CENÁRIO 2 & 3: Não aplicável. O tabuleiro está completo. ---

Lendo o tabuleiro do arquivo '/content/drive/MyDrive/trabalho-final-IA/tabuleiro9x9-valido.csv'...
Tabuleiro Lido:
 [[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]

==== ANÁLISE PARA O ARQUIVO: tabuleiro9x9-valido.csv ====

--- CENÁRIO 1: CLASSIFICAR TABULEIRO FECHADO (Questão 1) ---
Satisfação geral dos axio